In [11]:
import yfinance as yf
import datetime
from dateutil.relativedelta import relativedelta
from tqdm import tqdm
import numpy as np


In [33]:
class LoadData:

    def __init__(self, n_slide, data_range) -> None:
        self.train_data = np.ndarray(shape=(n_slide,data_range), dtype=float)
        self.test_data = np.ndarray(shape=(n_slide,data_range), dtype=float)
        self.date_data_start_list = []
        self.date_predict_start_list = []
        self.date_predict_end_list = []
        self.test_data_start_list = []

    def load_data(self, stock_name, date_predict_start, data_range, slide_range, n_slide):
        all_data = yf.Ticker(stock_name).history(period='max')
        all_data.index = all_data.index.date
        # all_data['Close'].iloc[0:len(all_data['Close'])] = list(range(0, len(all_data['Close'])))
        date_predict_start = self._check_start(date_predict_start, all_data)
        predict_start = all_data.index.get_loc(date_predict_start)
        for i in tqdm(range(n_slide)):
            predict_end = predict_start + data_range
            date_predict_end = all_data.iloc[predict_end].name
            data_start = predict_start - data_range
            date_data_start = all_data.iloc[data_start].name
            self.train_data[i, :] = all_data['Close'].iloc[data_start:predict_start].values
            self.test_data[i, :] = all_data['Close'].iloc[predict_start:predict_end].values
            data_start = data_start + slide_range
            predict_start = predict_start + slide_range
        return self.train_data, self.test_data

    def _check_start(self, date_predict_start, all_data):
        date_predict_start = datetime.datetime.strptime(
            date_predict_start, '%Y-%m-%d').date()
        check = date_predict_start in all_data.index
        while (check == False):
            date_predict_start = date_predict_start + \
                relativedelta(days=+1)
            check = date_predict_start in list(all_data.index)
        return date_predict_start
  

In [34]:
if __name__ == '__main__':
    stock_name = "^GSPC"
    date_predict_start = '2020-01-01'
    data_range = 10
    slide_range = 10
    n_slide = 5
    dataloader = LoadData(n_slide, data_range)
    train_data, test_data =\
        dataloader.load_data(stock_name, date_predict_start, data_range, slide_range, n_slide)

100%|██████████| 5/5 [00:00<00:00, 3570.84it/s]


In [35]:
train_data

array([[3192.52001953, 3191.13989258, 3205.37011719, 3221.2199707 ,
        3224.01000977, 3223.37988281, 3239.90991211, 3240.02001953,
        3221.29003906, 3230.7800293 ],
       [3257.85009766, 3234.85009766, 3246.2800293 , 3237.17993164,
        3253.05004883, 3274.69995117, 3265.35009766, 3288.12988281,
        3283.14990234, 3289.29003906],
       [3316.81005859, 3329.62011719, 3320.79003906, 3321.75      ,
        3325.54003906, 3295.4699707 , 3243.62988281, 3276.23999023,
        3273.39990234, 3283.65991211],
       [3225.52001953, 3248.91992188, 3297.59008789, 3334.68994141,
        3345.7800293 , 3327.70996094, 3352.09008789, 3357.75      ,
        3379.44995117, 3373.93994141],
       [3380.15991211, 3370.29003906, 3386.14990234, 3373.22998047,
        3337.75      , 3225.88989258, 3128.20996094, 3116.38989258,
        2978.76000977, 2954.2199707 ]])